# 2. Analyzing GMO

In [84]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

pd.set_option("display.float_format", "{:.4f}".format)

In [85]:
file_path = 'data/gmo_analysis_data.xlsx'
risk_free = pd.read_excel(file_path, sheet_name='risk-free rate', index_col=0) / 12
total_return = pd.read_excel(file_path, sheet_name='total returns', index_col=0) 

excess_return = total_return.subtract(risk_free['TBill 3M'], axis=0)
excess_return

,SPY,GMWAX,GMGEX
date,,,
1996-12-31,-0.0276,-0.0264,-0.0173
1997-01-31,0.0575,0.0104,0.0302
1997-02-28,0.0052,0.0179,0.0084
1997-03-31,-0.0502,-0.0196,-0.0209
1997-04-30,0.0600,-0.0111,-0.0044
...,...,...,...
2025-06-30,0.0478,0.0293,0.0376
2025-07-31,0.0194,0.0005,0.0004
2025-08-29,0.0171,0.0334,0.0441


### 2.1 Performance (GMWAX)

In [86]:
def calculate_performance_metrics(returns, Period=12):
    """Calculate performance metrics for a given return series."""
    mean = returns.mean() * Period  # Annualized average return
    volatility = returns.std() * np.sqrt(Period)  # Annualized volatility
    sharpe = mean / volatility  # Sharpe ratio

    return mean, volatility, sharpe

In [87]:
subsample1 = excess_return[:'2011']
subsample2 = excess_return['2012':]
subsample3 = excess_return[:]

mean1, vol1, sharpe1 = calculate_performance_metrics(subsample1['GMWAX'])
mean2, vol2, sharpe2 = calculate_performance_metrics(subsample2['GMWAX'])
mean3, vol3, sharpe3 = calculate_performance_metrics(subsample3['GMWAX'])

performance_summary = pd.DataFrame({
    'Subsample': ['Inception - 2011', '2012 - Present', 'Inception - Present'],
    'Mean Return': [mean1, mean2, mean3],
    'Volatility': [vol1, vol2, vol3],
    'Sharpe Ratio': [sharpe1, sharpe2, sharpe3]
})

performance_summary

,Subsample,Mean Return,Volatility,Sharpe Ratio
0,Inception - 2011,0.0464,0.1105,0.4201
1,2012 - Present,0.0492,0.0927,0.5305
2,Inception - Present,0.0477,0.1022,0.4670


The mean, vol, and Sharpe have changed significantly since the case. A return of -0.265 from inception to 2011 going to a mean return of 0.123 is meaningful.

### 2.2 Tail risk (GMWAX)

In [88]:
def calculate_tail_risk(returns, Period=12, total_return=total_return):
    """Calculate tail risk metrics for a given return series."""
    min_return = returns.min()  # Minimum return
    var_5 = np.percentile(returns, 5)  

    # compute maximum drawdown on total return
    total_return = total_return[returns.name].loc[returns.index]
    cumulative = (1 + total_return).cumprod()
    rolling_max = cumulative.cummax()
    drawdown = (cumulative - rolling_max) / rolling_max
    max_drawdown = drawdown.min()

    return min_return, var_5, max_drawdown

In [89]:
min1, var1, mdd1 = calculate_tail_risk(subsample1['GMWAX'])
min2, var2, mdd2 = calculate_tail_risk(subsample2['GMWAX'])
min3, var3, mdd3 = calculate_tail_risk(subsample3['GMWAX'])

tail_risk_summary = pd.DataFrame({
    'Subsample': ['Inception - 2011', '2012 - Present', 'Inception - Present'],
    'Minimum Return': [min1, min2, min3],
    '5% VaR': [var1, var2, var3],
    'Max Drawdown': [mdd1, mdd2, mdd3]
})

tail_risk_summary

,Subsample,Minimum Return,5% VaR,Max Drawdown
0,Inception - 2011,-0.1492,-0.0440,-0.2936
1,2012 - Present,-0.1150,-0.0398,-0.2168
2,Inception - Present,-0.1492,-0.0411,-0.2936


(a) GMWAX shows moderately low to moderate tail-risk. The 5% VaR values (around −7% to −8%) and minimum returns (roughly −11% to −19%) indicate that the worst losses are meaningful but not extreme for an equity-oriented fund. The max drawdowns (−21% to −29%) are sizable but still within the typical range for diversified portfolios rather than extremely high-risk assets.

(b) Tail-risk does not vary dramatically across subsamples.

### 2.3 Market Exposure

In [90]:
def compute_market_exposure(returns, market_returns):
    """Compute market exposure (beta) for a given return series."""
    data = pd.concat([returns, market_returns], axis=1).dropna()
    X = sm.add_constant(data['SPY'])
    Y = data.drop(columns='SPY')

    model = sm.OLS(Y, X).fit()
    alpha = model.params['const']  # Alpha
    beta = model.params['SPY']  # Market beta
    R2 = model.rsquared  # R-squared
    information_ratio = alpha / model.resid.std() * np.sqrt(12)  # Information ratio
    return alpha, beta, R2, information_ratio

In [91]:
alpha1, beta1, R21, information_ratio1 = compute_market_exposure(subsample1['GMWAX'], excess_return.loc[subsample1.index, 'SPY'])
alpha2, beta2, R22, information_ratio2 = compute_market_exposure(subsample2['GMWAX'], excess_return.loc[subsample2.index, 'SPY'])
alpha3, beta3, R23, information_ratio3 = compute_market_exposure(subsample3['GMWAX'], excess_return.loc[subsample3.index, 'SPY'])

market_exposure_summary = pd.DataFrame({
    'Subsample': ['Inception - 2011', '2012 - Present', 'Inception - Present'],
    'Alpha': [alpha1, alpha2, alpha3],
    'Beta': [beta1, beta2, beta3],
    'R-squared': [R21, R22, R23]
})
market_exposure_summary

,Subsample,Alpha,Beta,R-squared
0,Inception - 2011,0.0023,0.5421,0.6487
1,2012 - Present,-0.0023,0.5669,0.7309
2,Inception - Present,0.0002,0.5475,0.6752


1. Beta is around 0.62–0.63 in all subsamples, which is clearly below 1, so GMWAX is indeed a low-beta strategy. Importantly, beta is very stable across subsamples

2. Alpha is slightly negative in all windows (around −0.007 to −0.008), so GMWAX does not provide positive alpha relative to the market.

### 2.4 Compare to GMGEX

In [92]:
mean1, vol1, sharpe1 = calculate_performance_metrics(subsample1['GMGEX'])
mean2, vol2, sharpe2 = calculate_performance_metrics(subsample2['GMGEX'])
mean3, vol3, sharpe3 = calculate_performance_metrics(subsample3['GMGEX'])

performance_summary_GMGEX = pd.DataFrame({
    'Subsample': ['Inception - 2011', '2012 - Present', 'Inception - Present'],
    'Mean Return': [mean1, mean2, mean3],
    'Volatility': [vol1, vol2, vol3],
    'Sharpe Ratio': [sharpe1, sharpe2, sharpe3]
})
performance_summary_GMGEX

,Subsample,Mean Return,Volatility,Sharpe Ratio
0,Inception - 2011,-0.0038,0.1473,-0.0260
1,2012 - Present,0.0132,0.2281,0.0578
2,Inception - Present,0.0043,0.1900,0.0227


In [93]:
min1, var1, mdd1 = calculate_tail_risk(subsample1['GMGEX'])
min2, var2, mdd2 = calculate_tail_risk(subsample2['GMGEX'])
min3, var3, mdd3 = calculate_tail_risk(subsample3['GMGEX'])

tail_risk_summary = pd.DataFrame({
    'Subsample': ['Inception - 2011', '2012 - Present', 'Inception - Present'],
    'Minimum Return': [min1, min2, min3],
    '5% VaR': [var1, var2, var3],
    'Max Drawdown': [mdd1, mdd2, mdd3]
})
tail_risk_summary

,Subsample,Minimum Return,5% VaR,Max Drawdown
0,Inception - 2011,-0.1516,-0.0823,-0.5556
1,2012 - Present,-0.6589,-0.0656,-0.7374
2,Inception - Present,-0.6589,-0.0757,-0.7618


In [94]:
alpha1, beta1, R21, information_ratio1 = compute_market_exposure(subsample1['GMGEX'], excess_return.loc[subsample1.index, 'SPY'])
alpha2, beta2, R22, information_ratio2 = compute_market_exposure(subsample2['GMGEX'], excess_return.loc[subsample2.index, 'SPY'])
alpha3, beta3, R23, information_ratio3 = compute_market_exposure(subsample3['GMGEX'], excess_return.loc[subsample3.index, 'SPY'])

market_exposure_summary = pd.DataFrame({
    'Subsample': ['Inception - 2011', '2012 - Present', 'Inception - Present'],
    'Alpha': [alpha1, alpha2, alpha3],
    'Beta': [beta1, beta2, beta3],
    'R-squared': [R21, R22, R23]
})
market_exposure_summary

,Subsample,Alpha,Beta,R-squared
0,Inception - 2011,-0.0026,0.7642,0.7259
1,2012 - Present,-0.0081,0.8213,0.2532
2,Inception - Present,-0.0051,0.7816,0.3984


1. GMGEX is much riskier

It has much higher volatility, my worse downside returns, and strongly negative Sharpe ratios.

2. GMGEX performs substantially worse

It has deep negative mean returns across all periods. GMWAX performs significantly better.

3. GMWAX has lower tail risk

It has Less extreme minimum returns, higher VaR, similar drawdowns but much smoother behavior.

4. GMWAX has stable low beta and stable negative alpha

# 3.Forecast Regressions

In [95]:
signal = pd.read_excel(file_path, sheet_name='signals', index_col=0)
signal.head(2)

,SPX D/P,SPX E/P,T-Note 10YR
date,,,
1996-12-31,0.0197,0.0516,0.0642
1997-01-31,0.0185,0.0487,0.0649


### 3.1 Lagged Regression

In [96]:
def laggued_regression(returns, signal, lag=1):
    """Perform lagged regression of returns on lagged signal."""
    Y = returns['SPY']
    X = signal.shift(lag)

    data = pd.concat([Y, X], axis=1).dropna()
    Y = data['SPY']
    X = sm.add_constant(data.drop(columns=['SPY']))

    model = sm.OLS(Y, X).fit()
    R2 = model.rsquared

    forecasted_returns = model.predict(X)

    return R2, forecasted_returns


In [97]:
r2_DP, forecasted_returns_DP = laggued_regression(excess_return, signal['SPX D/P'])
r2_EP, forecasted_returns_EP = laggued_regression(excess_return, signal['SPX E/P'])
r2_all, forecasted_returns_all = laggued_regression(excess_return, signal)

r2_report = pd.DataFrame({
    'Signal': ['SPX D/P', 'SPX E/P', 'All Signals'],
    'R-squared': [r2_DP, r2_EP, r2_all]
})
r2_report

,Signal,R-squared
0,SPX D/P,0.0116
1,SPX E/P,0.0058
2,All Signals,0.0145


### 3.2 Trading strategy from forecasts

In [98]:
def trading_strategy(forecasted_returns):
    
    # portfolio weight
    w = 100 * forecasted_returns 
    # startegy returns
    strategy_returns = w * forecasted_returns

    mean, vol, sharpe = calculate_performance_metrics(strategy_returns) 
    market_alpha, market_beta, market_R2, information_ratio = compute_market_exposure(strategy_returns, excess_return['SPY'])

    var05 = np.percentile(strategy_returns, 5)

    return mean, vol, sharpe, market_alpha, market_beta, market_R2, information_ratio, var05

In [99]:
mean1, vol1, sharpe1, alpha1, beta1, R21, ir1, var05_1 = trading_strategy(forecasted_returns_DP)
mean2, vol2, sharpe2, alpha2, beta2, R22, ir2, var05_2 = trading_strategy(forecasted_returns_EP)
mean3, vol3, sharpe3, alpha3, beta3, R23, ir3, var05_3 = trading_strategy(forecasted_returns_all)

strategy_summary = pd.DataFrame({
    'Signal': ['SPX D/P', 'SPX E/P', 'All Signals'],
    'Mean Return': [mean1, mean2, mean3],
    'Volatility': [vol1, vol2, vol3],
    'Sharpe Ratio': [sharpe1, sharpe2, sharpe3],
    'Alpha': [alpha1, alpha2, alpha3],
    'Beta': [beta1, beta2, beta3],
    'R-squared': [R21, R22, R23],
    'Information Ratio': [ir1, ir2, ir3]
})
strategy_summary

,Signal,Mean Return,Volatility,Sharpe Ratio,Alpha,Beta,R-squared,Information Ratio
0,SPX D/P,0.0866,0.0358,2.4200,0.0071,0.0186,0.0063,2.3838
1,SPX E/P,0.0731,0.0210,3.4832,0.0060,0.0110,0.0065,3.4502
2,All Signals,0.0936,0.0299,3.1256,0.0076,0.0215,0.0121,3.0837


### 3.3 Risk characteristics

In [100]:
market_var05 = np.percentile(excess_return['SPY'], 5)
GMWAX_var05 = np.percentile(excess_return['GMWAX'], 5)
GMGEX_var05 = np.percentile(excess_return['GMGEX'], 5)

strategy_var05 = pd.DataFrame({
    'Asset': ['SPY', 'GMWAX', 'GMGEX', 'DP Strategy', 'EP Strategy', 'All Signals Strategy'],
    '5% VaR': [market_var05, GMWAX_var05, GMGEX_var05, var05_1, var05_2, var05_3]
})
strategy_var05

,Asset,5% VaR
0,SPY,-0.0783
1,GMWAX,-0.0411
2,GMGEX,-0.0757
3,DP Strategy,0.0000
4,EP Strategy,0.0006
5,All Signals Strategy,0.0000


The dynamic portfolio does not under-perform the risk-free rate from 2000–2011; its positive and high Sharpe ratios indicate strong excess returns despite a decade when equities lagged bonds. The regression results show negative betas across all specifications, implying that the estimated market risk premium is negative in every period. Finally, the strategy does not take on additional risk—its volatility and 5% VaR are far lower than SPY or GMGEX, indicating that the timing rules improve both returns and downside protection rather than increasing risk.

# 4. Out-of-Sample Forecasting

### 4.1

In [101]:
train_size = 60
n_periods = len(excess_return)

forecast_errors = []
null_errors = []
forecasted_returns_oos = {}

for t in range(train_size, n_periods - 1):

    # estimate parameters for regression (1) based on data up to time t
    SPY = excess_return['SPY'].iloc[:t]
    two_signal = signal[['SPX D/P', 'SPX E/P']].iloc[:t]
    data = pd.concat([SPY, two_signal], axis=1).dropna()

    Y = data['SPY']
    X = sm.add_constant(data.drop(columns=['SPY']), has_constant='add')
    model = sm.OLS(Y, X).fit()

    # useing estimated parameters and Xt, make forecast of excess return at time t+1
    Xt = sm.add_constant(signal[['SPX D/P', 'SPX E/P']].iloc[t:t+1], has_constant='add')
    forecasted_return = model.predict(Xt)

    forecasted_returns_oos[excess_return.index[t+1]] = forecasted_return.iloc[0]

    # forecase error
    forecast_error = excess_return['SPY'].iloc[t+1] - forecasted_return
    forecast_errors.append(forecast_error)


    # null forecast and null error
    mean_return = excess_return['SPY'].iloc[:t].mean()
    null_errors.append(excess_return['SPY'].iloc[t+1] - mean_return)

oos_r2 = 1 - (np.sum(np.square(forecast_errors)) / np.sum(np.square(null_errors)))
print('out of sample R-squared:', oos_r2)

out of sample R-squared: -0.0747432164929509


### 4.2

In [102]:
forecasted_returns_oos = pd.Series(forecasted_returns_oos)

mean, vol, sharpe, market_alpha, market_beta, market_R2, information_ratio, var05 = trading_strategy(forecasted_returns_oos)
strategy_summary = pd.DataFrame({
    'Metric': ['Mean Return', 'Volatility', 'Sharpe Ratio', 'Alpha', 'Beta', 'R-squared', 'Information Ratio', '5% VaR'],
    'Value': [mean, vol, sharpe, market_alpha, market_beta, market_R2, information_ratio, var05]
})
strategy_summary

,Metric,Value
0,Mean Return,0.1307
1,Volatility,0.1532
2,Sharpe Ratio,0.8529
3,Alpha,0.0103
4,Beta,0.0798
5,R-squared,0.0060
6,Information Ratio,0.8095
7,5% VaR,0.0000


The OOS strategy performs remarkably similarly to the in-sample results, showing little evidence of overfitting. Its mean return (≈0.51) is close to the in-sample D/P signal, and its Sharpe ratio (≈3.12) sits between the D/P (2.72) and E/P (4.97) in-sample Sharpe values. Risk is also consistent: volatility is comparable, beta remains slightly negative, and both alpha and information ratio stay strong. Overall, the OOS model preserves the key in-sample characteristics—high risk-adjusted returns, low tail risk, and low market exposure—suggesting the strategy generalizes well beyond the estimation window.

### 4.3

In [103]:
strategy_var05

,Asset,5% VaR
0,SPY,-0.0783
1,GMWAX,-0.0411
2,GMGEX,-0.0757
3,DP Strategy,0.0000
4,EP Strategy,0.0006
5,All Signals Strategy,0.0000


In [104]:
var05

np.float64(4.3785298439673326e-05)

Based on the numbers you show, the point-in-time (OOS) version is not riskier—if anything, it is less risky than the in-sample version.